<a href="https://colab.research.google.com/github/Debashish-hub/Covid-detection-using-chest-X-ray-Images/blob/main/2_Covid_19_Detection_Using_X_ray_Images_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Dataset link : http://cb.lk/covid_19 (From the data we prepared)

!wget http://cb.lk/covid_19


--2021-11-11 05:24:39--  http://cb.lk/covid_19
Resolving cb.lk (cb.lk)... 104.21.72.203, 172.67.187.81, 2606:4700:3031::6815:48cb, ...
Connecting to cb.lk (cb.lk)|104.21.72.203|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cb.lk/covid_19 [following]
--2021-11-11 05:24:40--  https://cb.lk/covid_19
Connecting to cb.lk (cb.lk)|104.21.72.203|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0 [following]
--2021-11-11 05:24:40--  https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip [following]
--2021-11

In [ ]:
!unzip covid_19  #Unzipping data

Archive:  covid_19
replace CovidDataset/Val/Covid/88de9d8c39e946abd495b37cd07d89e5-6531-0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: CovidDataset/Val/Covid/88de9d8c39e946abd495b37cd07d89e5-6531-0.jpg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-8.jpg  
  inflating: CovidDataset/Val/Covid/85E52EB3-56E9-4D67-82DA-DEA247C82886.jpeg  
  inflating: CovidDataset/Val/Covid/4-x-day1.jpg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-53.jpg  
  inflating: CovidDataset/Val/Covid/4e43e48d52c9e2d4c6c1fb9bc1544f_jumbo.jpeg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-35-1.jpg  
  inflating: CovidDataset/Val/Covid/53EC07C9-5CC6-4BE4-9B6F-D7B0D72AAA7E.jpeg  
  inflating: CovidDataset/Val/Covid/6CB4EFC6-68FA-4CD5-940C-BEFA8DAFE9A7.jpeg  
  inflating: CovidDataset/Val/Covid/58cb9263f16e94305c730685358e4e_jumbo.jpeg  
  inflating: CovidDataset/Val/Covid/6b3bdbc31f65230b8cdcc3cef5f8ba8a-40ac-0.jpg  
  inflating: CovidDataset/Val/Covid/auntminnie-d-2020_01_

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from keras.models import Sequential
from keras.preprocessing import image

In [ ]:
train_datagen = image.ImageDataGenerator(
    rescale = 1/255, 
    horizontal_flip = True, 
    zoom_range = 0.2, 
    shear_range = 0.2
)

train_data = train_datagen.flow_from_directory(directory="/content/CovidDataset/Train", 
                                               target_size=(256,256), 
                                               batch_size=16, 
                                               class_mode = 'binary'
                                               )

Found 224 images belonging to 2 classes.


In [ ]:
train_data.class_indices

{'Covid': 0, 'Normal': 1}

In [ ]:
test_datagen = image.ImageDataGenerator(
    rescale = 1/255
)

test_data = test_datagen.flow_from_directory(directory="/content/CovidDataset/Val", 
                                             target_size=(256,256), 
                                             batch_size=16, 
                                             class_mode = 'binary'
                                             )

Found 60 images belonging to 2 classes.


In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = (256, 256, 3)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units = 64, activation= 'relu'))
model.add(Dropout(rate = 0.50))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss= keras.losses.binary_crossentropy, optimizer= "adam", metrics = ['acc'])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 conv2d_10 (Conv2D)          (None, 252, 252, 64)      18496     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 126, 126, 64)     0         
 2D)                                                             
                                                                 
 dropout_9 (Dropout)         (None, 126, 126, 64)      0         
                                                                 
 conv2d_11 (Conv2D)          (None, 124, 124, 128)     73856     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 62, 62, 128)      0         
 2D)                                                  

In [ ]:
model.fit_generator(train_data, steps_per_epoch=8, epochs=10, validation_steps=2, validation_data= test_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/10
8/8 [==============================] - 9s 1s/step - loss: 2.1380 - acc: 0.5547 - val_loss: 0.6878 - val_acc: 0.8438
Epoch 2/10
8/8 [==============================] - 7s 918ms/step - loss: 0.6166 - acc: 0.7500 - val_loss: 0.4898 - val_acc: 0.9375
Epoch 3/10
8/8 [==============================] - 8s 1s/step - loss: 0.4245 - acc: 0.8203 - val_loss: 0.3674 - val_acc: 0.8750
Epoch 4/10
8/8 [==============================] - 8s 988ms/step - loss: 0.4026 - acc: 0.8203 - val_loss: 0.3045 - val_acc: 0.9062
Epoch 5/10
8/8 [==============================] - 7s 949ms/step - loss: 0.3378 - acc: 0.8750 - val_loss: 0.2558 - val_acc: 0.9062
Epoch 6/10
8/8 [==============================] - 8s 960ms/step - loss: 0.2693 - acc: 0.8828 - val_loss: 0.1681 - val_acc: 0.9375
Epoch 7/10
8/8 [==============================] - 8s 1s/step - loss: 0.1692 - acc: 0.9062 - val_loss: 0.1149 - val_acc: 0.9688
Epoch 8/10
8/8 [==============================] - 8s 922ms/step - loss: 0.2367 - acc: 0.9297 - val_

In [ ]:
path = "/content/CovidDataset/Val/Normal/NORMAL2-IM-0535-0001-0002.jpeg"
img = image.load_img(path, target_size=(256,256))

img = image.img_to_array(img)/255
img = np.array([img])
img.shape

(1, 256, 256, 3)

In [ ]:
(model.predict(img) > 0.5).astype("int32")

array([[1]], dtype=int32)